Looking at Metrics and Plots Made From the Command Line for the pipeline matchedVisitQualityCore
-------------------------------------------------------------------------------------------------

- SDC : update 2024-03-08
- SDC : update 2024-03-14
-  SDC : update 2024-03-15 : w_2024_10

### Links

- Plot Navigator : https://usdf-rsp.slac.stanford.edu/plot-navigator/dashboard_gen3
- Doc on pipelines : https://pipelines.lsst.io/v/daily/modules/lsst.analysis.tools/index.html
- Code on GitHub : https://github.com/lsst/analysis_tools


- **the pipeline** : https://github.com/lsst/analysis_tools/blob/main/pipelines/matchedVisitQualityCore.yaml
- **the command** : https://github.com/sylvielsstfr/analysis_tools-auxtel/blob/main/scripts_run_pipelines/run-pipe-matchedVisitQualityCore.sh

        pipetask run -p $ANALYSIS_TOOLS_DIR/pipelines/matchedVisitQualityCore.yaml \
        -b /sdf/group/rubin/repo/oga \
        -i LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240201/w_2024_05/PREOPS-4871 \
        -o u/dagoret/analysis_tools_test/Plots_matchedVisitQualityCore \
        -d "instrument='LATISS' AND (band='g' or band='r' or band='i' or band='z' or band='y') AND skymap='latiss_v1' \
        AND tract=3864 AND (patch=236 or patch=237)" \
        --register-dataset-types --prune-replaced=purge --replace-run

Following the getting started guide we have run a pipeline from the command line to produce the coadd quality core plots and metrics.

In [1]:
from lsst.analysis.tools.tasks.reconstructor import reconstructAnalysisTools
import lsst.daf.butler as dafButler
from lsst.analysis.tools.interfaces._task import _StandinPlotInfo

import numpy as np

In [2]:
skymap_sel = "latiss_v1"
instrument_sel = "LATISS"
tract_sel = 3864
patch_sel = 236 # or 237
band_sel = "g"
table_sel = "objectTable_tract" # not for 

# plotting range for visits
XLIM = (122.3,122.7)
YLIM = (-36.3,-36.05)

Let's read back in the data that we just made

In [3]:
#collection = "/sdf/group/rubin/u/sr525/coaddQualityCoreDemoRepo"
#collection ="LATISS/runs/AUXTEL_DRP_IMAGING_2023-11A-10A-09AB-08ABC-07AB-05AB/w_2023_46/PREOPS-4553"
#collection = "LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240201/w_2024_05/PREOPS-4871"
#collection = "LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240311/w_2024_10/PREOPS-4985"
collection = "u/dagoret/analysis_tools_test/Plots_matchedVisitQualityCore"
repo = '/sdf/group/rubin/repo/oga/'
dataId = {"instrument":instrument_sel,"tract": tract_sel, "skymap": skymap_sel}
Butler = dafButler.Butler(repo)
skymap = Butler.get('skyMap', skymap=skymap_sel, collections=collection)

In [4]:
dataId

{'instrument': 'LATISS', 'tract': 3864, 'skymap': 'latiss_v1'}

The reconstructor contains all of the plots/metrics that we just made, we specify the label and that gives us just the ones in that section of the pipeline.

In [5]:
anaList = list(Butler.registry.queryDatasetTypes('*ana*'))
# Note: please ignore the Spectractor WARNING messages below, they are to be expected here.
for ana in anaList:
    print(ana)

DatasetType('analyzeSourceTableCore_config', {}, Config)
DatasetType('analyzeSourceTableCore_metadata', {band, instrument, detector, physical_filter, visit}, TaskMetadata)
DatasetType('analyzeSourceTableCore_log', {band, instrument, detector, physical_filter, visit}, ButlerLogRecords)
DatasetType('analyzeMatchedVisitCore_config', {}, Config)
DatasetType('analyzeSourceTableCoreDevel_config', {}, Config)
DatasetType('analyzeSourceTableCoreDevel_metadata', {band, instrument, physical_filter, visit}, TaskMetadata)
DatasetType('analyzeSourceTableCoreDevel_log', {band, instrument, physical_filter, visit}, ButlerLogRecords)
DatasetType('analyzeMatchedVisitQualityCore_config', {}, Config)
DatasetType('analyzeMatchedVisitQualityCore_metadata', {instrument, skymap, tract}, TaskMetadata)
DatasetType('analyzeMatchedVisitQualityCore_log', {instrument, skymap, tract}, ButlerLogRecords)
DatasetType('analyzeMatchedVisitCore_metadata', {instrument, skymap, tract}, TaskMetadata)
DatasetType('analyzeMatc

In [6]:
dataId

{'instrument': 'LATISS', 'tract': 3864, 'skymap': 'latiss_v1'}

### pipeline file description
-**the pipeline** : https://github.com/lsst/analysis_tools/blob/main/pipelines/matchedVisitQualityCore.yaml

    analysis_tools/pipelines/matchedVisitQualityCore.yaml
     description: |
     Tier1 plots and metrics to assess repeatability of sources matched per tract
     tasks:
        analyzeMatchedVisitCore:
        class: lsst.analysis.tools.tasks.AssociatedSourcesTractAnalysisTask
        config:
          connections.outputName: matchedVisitCore
          atools.stellarPhotometricRepeatability: StellarPhotometricRepeatability
          atools.stellarPhotometricResiduals: StellarPhotometricResidualsFocalPlane
          atools.stellarAstrometricResidualsRA: StellarAstrometricResidualsRAFocalPlanePlot
          atools.stellarAstrometricResidualsDec: StellarAstrometricResidualsDecFocalPlanePlot
          atools.stellarAstrometricResidualStdDevRA: StellarAstrometricResidualStdDevRAFocalPlanePlot
          atools.stellarAstrometricResidualStdDevDec: StellarAstrometricResidualStdDevDecFocalPlanePlot
          atools.stellarAstrometricRepeatability1: AstrometricRelativeRepeatability
          atools.stellarAstrometricRepeatability1.xValue: 1
          atools.stellarAstrometricRepeatability1.process.calculateActions.rms.annulus: 5
          atools.stellarAstrometricRepeatability2: AstrometricRelativeRepeatability
          atools.stellarAstrometricRepeatability2.xValue: 2
          atools.stellarAstrometricRepeatability2.process.calculateActions.rms.annulus: 20
          atools.stellarAstrometricRepeatability3: AstrometricRelativeRepeatability
          atools.stellarAstrometricRepeatability3.xValue: 3
          atools.stellarAstrometricRepeatability3.process.calculateActions.rms.annulus: 200
          atools.stellarAstrometricRepeatability3.process.calculateActions.rms.threshAD: 30

In [7]:
taskState, inputData = reconstructAnalysisTools(Butler, 
                                                collection=collection,
                                                #label="analyzeObjectTableCore",
                                                #label="analyzeMatchedVisitQualityCore",
                                                
                                                label="analyzeMatchedVisitCore",
                                                #label="matchedVisitCore",
                                                dataId=dataId, 
                                                callback=None
)

/home/d/dagoret/repos/repos_w_2024_10/analysis_tools/python/lsst/analysis/tools/tasks/reconstructor.py:91: UserWarning: Collections should not be specified with DatasetRef
  container.append(butler.get(ref, collections=(collection,)))
/home/d/dagoret/repos/repos_w_2024_10/analysis_tools/python/lsst/analysis/tools/tasks/reconstructor.py:91: UserWarning: Collections should not be specified with DatasetRef
  container.append(butler.get(ref, collections=(collection,)))
/home/d/dagoret/repos/repos_w_2024_10/analysis_tools/python/lsst/analysis/tools/tasks/reconstructor.py:91: UserWarning: Collections should not be specified with DatasetRef
  container.append(butler.get(ref, collections=(collection,)))
/home/d/dagoret/repos/repos_w_2024_10/analysis_tools/python/lsst/analysis/tools/tasks/reconstructor.py:91: UserWarning: Collections should not be specified with DatasetRef
  container.append(butler.get(ref, collections=(collection,)))
/home/d/dagoret/repos/repos_w_2024_10/analysis_tools/python/

### Inputs to the aTools

- **the analysis tool task** : https://github.com/lsst/analysis_tools/blob/main/python/lsst/analysis/tools/tasks/associatedSourcesTractAnalysis.py
- **the atool** : https://github.com/lsst/analysis_tools/blob/main/python/lsst/analysis/tools/atools/photometricRepeatability.py

In [8]:
inputData.keys()

dict_keys(['skyMap', 'associatedSources', 'sourceCatalogs'])

In [9]:
#inputData['sourceCatalogs'][:2] 

Let's list the actions that were run

In [10]:
for action in taskState.atools:
    print(action.identity)

stellarAstrometricRepeatability1
stellarAstrometricRepeatability2
stellarAstrometricRepeatability3
stellarAstrometricResidualStdDevDec
stellarAstrometricResidualStdDevRA
stellarAstrometricResidualsDec
stellarAstrometricResidualsRA
stellarPhotometricRepeatability
stellarPhotometricResiduals


Now we can recreate them, we're going to pick one and look at it.

In [11]:
band_sel

'g'

In [12]:
bands_dict = dict(band=band_sel)
bands_dict 

{'band': 'g'}

In [30]:
taskState.atools.stellarPhotometricRepeatability.prep.toDict()["selectors"]

{'bandSelector': {'vectorKey': 'band', 'bands': ['g']}}

In [29]:
taskState.atools.stellarPhotometricRepeatability.prep.selectors.bandSelector.bands = ['g']

In [31]:
taskState.atools.stellarPhotometricRepeatability.finalize()

In [22]:
dir(taskState.atools.stellarPhotometricRepeatability.prep.toDict)

['__annotations__',
 '__call__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_collectImports',
 '_fields',
 '_fromPython',
 '_frozen',
 '_history',
 '_imports',
 '_name',
 '_rename',
 '_save',
 '_source',
 '_storage',
 'addInputSchema',
 'applyContext',
 'compare',
 'formatHistory',
 'freeze',
 'getFormattedInputSchema',
 'getInputSchema',
 'getOutputSchema',
 'history',
 'identity',
 'items',
 'keys',
 'keysToLoad',
 'load',
 'loadFromStream',
 'loadFromString',
 'names',
 'save',
 'saveToStream',
 'saveToString',
 'selectors',
 'setDefaults',
 'toDict',
 'update',
 'validate',
 'values',
 'vectorKeys']

In [32]:
plotInfoDict = {"run": collection, "bands": band_sel, "tract": tract_sel, "tableName": table_sel}
#taskState.atools.stellarAstrometricRepeatability1(inputData, plotInfo=plotInfoDict, skymap=skymap, band=band_sel)

taskState.atools.stellarPhotometricRepeatability(inputData, plotInfo=plotInfoDict, skymap=skymap)

#taskState.atools.stellarAstrometricRepeatability1(inputData, plotInfo=plotInfoDict, skymap=skymap )

KeyError: 'band'

In [33]:
taskState.atools.stellarPhotometricResiduals(inputData, plotInfo=plotInfoDict, skymap=skymap, band=band_sel)

KeyError: 'band'

Let's see the selectors that were used

In [ ]:
for selector in taskState.atools.e1Diff.prep.selectors:
    print(selector.identity)

We can look at each step individually

In [ ]:
prep = taskState.atools.e1Diff.prep(inputData["data"], plotInfo=plotInfoDict, skymap=skymap, band=band_sel)
#print("Prep:", prep)
process = taskState.atools.e1Diff.process(prep, band=band_sel)
#print("Process:", process)
produce = taskState.atools.e1Diff.produce(process, band=band_sel, plotInfo=plotInfoDict, skymap=skymap)
#print("Produce:", produce)

Let's change something about the selectors and then re run the plot

In [ ]:
taskState.atools.e1Diff.prep.selectors.snSelector.threshold = 10
prep = taskState.atools.e1Diff.prep(inputData["data"], plotInfo=plotInfoDict, skymap=skymap, band=band_sel)
process = taskState.atools.e1Diff.process(prep, band=band_sel)
produce = taskState.atools.e1Diff.produce(process, band=band_sel, plotInfo=plotInfoDict, skymap=skymap)

In [ ]:
taskState.atools.e2Diff(inputData["data"], plotInfo=plotInfoDict, skymap=skymap, band=band_sel)

## shapeSizeFractionalDiff

In [ ]:
taskState.atools.shapeSizeFractionalDiff(inputData["data"], plotInfo=plotInfoDict, skymap=skymap, band=band_sel)

## skyFluxStatisticMetric

In [ ]:
taskState.atools.skyFluxStatisticMetric(inputData["data"], plotInfo=plotInfoDict, skymap=skymap, band=band_sel)

## skyObjectFlux

In [ ]:
taskState.atools.skyObjectFlux(inputData["data"], plotInfo=plotInfoDict, skymap=skymap, band=band_sel)

## skyObjectSky

In [ ]:
taskState.atools.skyObjectSky(inputData["data"], plotInfo=plotInfoDict, skymap=skymap, band=band_sel)

## wPerpCModel

In [ ]:
taskState.atools.wPerpCModel(inputData["data"], plotInfo=plotInfoDict, skymap=skymap, band=band_sel)

## wPerpPSFP

In [ ]:
taskState.atools.wPerpPSFP(inputData["data"], plotInfo=plotInfoDict, skymap=skymap, band=band_sel)

## xPerpCModel

In [ ]:
taskState.atools.xPerpCModel(inputData["data"], plotInfo=plotInfoDict, skymap=skymap, band=band_sel)

## xPerpPSFP

In [ ]:
taskState.atools.xPerpPSFP(inputData["data"], plotInfo=plotInfoDict, skymap=skymap, band=band_sel)
